# Resolução de sistemas de equações lineares

## Importações

In [1]:
import numpy as np

## Funções Auxiliares

In [2]:
def pivoting_function(A, B, base, length):
  # choose pivot
  max = (abs(A[base, base]), base)
  for row in range(base, length - 1):
    if abs(A[row + 1, 0]) > max[0]:
      max = (A[row + 1, 0], row + 1)

  if max[0] != base:
    aux = np.copy(A[base])
    A[base] = A[max[1]]
    A[max[1]] = aux

    aux = np.copy(B[base])
    B[base] = B[max[1]]
    B[max[1]] = aux

In [96]:
def unicity(matrix):
  A = np.copy(matrix)
  A_1 = np.array(A[0,0])
  A_2 = np.delete(np.delete(A, 0, axis=1), 0, axis=0)

  if np.linalg.det(A) != 0 and np.linalg.det(A_2) != 0:
    return True

  print("Sistema não tem solução única")
  print("Determinante da matriz é igual a 0")
  return False

## Definições

In [106]:
# Ex1

A1 = np.array([
          [0, 1, 3, 2, 4],
          [8, 2, 9, -1, 2],
          [5, 1, 1, 7, 2],
          [-2, 4, 5, 1, 0],
          [7, -3, 2, -4, 1]
          ], dtype='float')  

B1 = np.array([3, -5, 6, -1, 8], dtype='float')

# Ex2

A2 = np.array([
          [2, 2, 1],
          [3, 3, 2],
          [3, 2, 1]
          ], dtype='float')  
          
A3 = np.array([
          [3, 2, 1],
          [2, 2, 1],
          [3, 3, 1]
          ], dtype='float')  

A4 = np.array([
          [2, 1, 3],
          [4, 3, 8],
          [6, 7, 17]
          ], dtype='float')  

# Ex3

A5 = np.array([
          [1, 2, 0, -3, 3],
          [2, 5, -1, 1, 4],
          [-3, -1, 50, 1, -19],
          [0, 1, 1, 6, 0],
          [3, 4, -19, 0, 39]
          ], dtype='float')  

B5 = np.array([17, 41, -45, 30, 51], dtype='float')

# Ex4

A6 = np.array([
          [1, -0.5, 0.5],
          [1, 1, 1],
          [-0.5, -0.5, 1]], 
          dtype='float')

B6 = np.array([3, 12, 3], dtype='float')

A7 = np.array([
          [10, 2, -3, 5],
          [1, 8, -1, 2],
          [2, -1, -5, 1],
          [-1, 2, 3, 20]],
          dtype='float')

B7 = np.array([48, 4, -11, 150], dtype='float')

## Algoritmos

### Eliminação de Gauss

In [75]:
def gauss_elimination(coeficients, constants, precision=8, pivoting=False, factors=False):
  A = np.copy(coeficients)
  B = np.copy(constants)
  length = len(A) - 1

  factors_list = np.identity(len(A), dtype='float')

  # superior matrix
  for base in range(length):
    if pivoting:
      pivoting_function(A, B, base, length)
      
    for row in range(base + 1, len(A)):
      # pivot validation
      if A[base, base] == 0 and not pivoting:
        print('Não foi possível montar a matriz superior, pivô nulo!')
        print(f'linha: {base}, coluna: {base}')
        return None

      factor = A[row, base] / A[base, base]
      factors_list[row, base] = factor
      
      A[row, base] = 0

      for col in range(base + 1, len(A)):
        A[row, col] = A[row, col] - factor * A[base, col]

      B[row] = B[row] - (factor * B[base])

  # resolution
  x = np.zeros(len(A), dtype='float')

  if A[length, length] == 0:
    print('Não foi possível resolver o sistema, divisão por zero!')
    print(f'linha: {length}, coluna: {length}')
    return None

  x[length] = B[length] / A[length, length]

  for row in range(length, 0, -1):
    s = 0
    for col in range(row + 1, len(A)):
      s = s + A[row, col] * x[col]
      x[row] = (B[row] - s) / A[row, row]

  np.set_printoptions(precision=precision)

  if factors:
    return A, B, x, factors_list
  else:
    return A, B, x


### Fatoração LU

In [98]:
def lu_decomposition(coeficients):
  A = np.copy(coeficients)
  length = len(A) - 1

  if not unicity(A):
    return None

  gauss = gauss_elimination(A, np.zeros(len(A), dtype='float'), factors=True)

  if(gauss == None):
    return None
    
  lower = gauss[0]
  upper = gauss[3]

  return lower, upper

### Fatoração de Cholesky

In [102]:
def cholesky(coeficients, constants):
  A = np.copy(coeficients)
  B = np.copy(constants)

  length = len(A) - 1

  if not unicity(A):
    return None

### Jacobi

In [123]:
def jacobi(coeficients, constants, precision=8, iterations=100):
  A = np.copy(coeficients)
  B = np.copy(constants)

  x = np.zeros(len(A), dtype='float')
  v = np.zeros(len(A), dtype='float')

  np.set_printoptions(precision=5)

  for i in range(len(A) - 1):
    x[i] = B[i] / A[i, i]

  iter = 0
  
  while True:
    iter += 1
    
    for i in range(len(A) - 1):
      sum = 0

      for j in range(len(A) - 1):
        if i != j:
          sum += A[i, j] * x[j]
      
      v[i] = (B[i] - sum) / A[i, i]

    norma_num = 0
    norma_den = 0

    for i in range(len(A) - 1):
      t = abs(v[i] - x[i])

      if t > norma_num:
        norma_num = t

      if abs(v[i]) > norma_den:
        norma_den = abs(v[i])
      
      x[i] = v[i]

    norma_rel = norma_num / norma_den

    print(iter, x)

    if norma_rel < precision or iter > 100:
      break

  if norma_rel <= precision:
    info = 0
  else:
    info = 1
    
  return x, iter, info

## Exercícios

### Exercício 1 

Resolva o SELA pelo método de Eliminação de Gauss sem pivotamento e com pivotamento parcial, retendo, durante as eliminações, cinco algarismos depois da vírgula. Compare os resultados. (A1, B1)

In [10]:
gauss_elimination(A1, B1, precision=5, pivoting=False)

Não foi possível resolver o sistema, pivô nulo!
linha: 0, coluna: 0


In [64]:
gauss_elimination(A1, B1, precision=5, pivoting=True)

(array([[  8.     ,   2.     ,   9.     ,  -1.     ,   2.     ],
        [  0.     ,   1.     ,   3.     ,   2.     ,   4.     ],
        [  0.     ,   0.     ,  -3.875  ,   8.125  ,   1.75   ],
        [  0.     ,   0.     ,   0.     , -21.35484, -20.32258],
        [  0.     ,   0.     ,   0.     ,   0.     ,  -0.74622]]),
 array([ -5.     ,   3.     ,   9.875  , -31.67742,  12.46224]),
 array([  0.     , -43.98381,  26.34413,  17.37652, -16.7004 ]))

#### Conclusão

Podemos verificar que, com o sistema linear fornecido pelo exercício, podemos apenas resolver utilizando a eliminação de Gauss com pivotamento pois se enquadra exatamente em um das motivações para se utilizar o pivotamento, que seria o pivô escolhido, nesse caso o primeiro elemento da primeira coluna, acabar resultando em uma divisão por zero, o que inviabiliza a comparação das duas variações

### Exercício 2

Quais das matrizes (A2, A3, A4) podem ser decompostas na forma LU ? Decompor as que forem possíveis.

In [100]:
lu_decomposition(A2)

Não foi possível montar a matriz superior, pivô nulo!
linha: 1, coluna: 1


In [99]:
lu_decomposition(A3)

(array([[ 3.        ,  2.        ,  1.        ],
        [ 0.        ,  0.66666667,  0.33333333],
        [ 0.        ,  0.        , -0.5       ]]),
 array([[1.        , 0.        , 0.        ],
        [0.66666667, 1.        , 0.        ],
        [1.        , 1.5       , 1.        ]]))

In [101]:
lu_decomposition(A4)

Não foi possível resolver o sistema, divisão por zero!
linha: 2, coluna: 2


#### Conclusão

Utilizando o método de decomposição LU com passos intermediários da eliminação de Gauss sem pivotamento, foi possível decompor apenas a matriz B (A3), pois na matriz A (A2), tivemos uma divisão por zero no momento da montagem da matriz superior e na matriz C (A4) também houve divisão por zero, porém no momento da resolução do sistema.

### Exercício 3

Resolva o SELA pelo método de Fatoração de Cholesky, retendo, durante as eliminações, cinco algarismos depois da vírgula.

### Exercício 4

Resolva-os usando os métodos de Gauss-Jacobi e Gauss-Seidel e verifique a convergência dos métodos. Justifique os resultados obtidos.

In [127]:
jacobi(A6, B6, precision=0.01, iterations=5)

1 [9. 9. 0.]
2 [7.5 3.  0. ]
3 [4.5 4.5 0. ]
4 [5.25 7.5  0.  ]
5 [6.75 6.75 0.  ]
6 [6.375 5.25  0.   ]
7 [5.625 5.625 0.   ]
8 [5.8125 6.375  0.    ]
9 [6.1875 6.1875 0.    ]
10 [6.09375 5.8125  0.     ]
11 [5.90625 5.90625 0.     ]
12 [5.95312 6.09375 0.     ]
13 [6.04688 6.04688 0.     ]
14 [6.02344 5.95312 0.     ]
15 [5.97656 5.97656 0.     ]


(array([5.97656, 5.97656, 0.     ]), 15, 0)